Import necessary libraries:

In [15]:
import json
import os
import copy
from metadataSchemaReader import MetadataSchemaReader
from attributeMapping import AttributeMapping
import datetime as dt
from dateutil import parser
import logging

# view all the metadata stored in the ZEISS TIFF file
# from https://github.com/ks00x/zeiss_tiff_meta.git
import zeiss_tiff_meta.zeisstiffmeta as zm

Read in source image and extract metadata using zeiss method:

In [3]:
src = 'FeMoOx_AntiA_04_1k5x_CN.tif' # Read in source image
tab = zm.zeiss_meta(src)            # Extract the full metadata

Clean up the metadata:

In [4]:
del tab[0] # delete first result as it is useless

# 
resultDictionary = {**dict((x1+"_value",x2) for x0, x1, x2, x3 in tab) , **dict((x1+"_unit",x3) for x0, x1, x2, x3 in tab)}
resultDictionary_filtered={k: v for k, v in resultDictionary.items() if v != ''}
resultDictionary_filtered_sorted=dict(sorted(resultDictionary_filtered.items()))
resultDictionary = dict((x1, {"value":x2,"unit":x3}) for x0, x1, x2, x3 in tab)
# resultDictionary_filtered_sorted['Store resolution_value'].split('*')
resultDictionary_filtered_sorted

{'ASAP DFE1 Minus5V Analogue_unit': 'V',
 'ASAP DFE1 Minus5V Analogue_value': 0.0,
 'ASAP DFE1 Plus5V Analogue_unit': 'V',
 'ASAP DFE1 Plus5V Analogue_value': 0.0,
 'ASAP DFE1 Plus5V Digital_unit': 'V',
 'ASAP DFE1 Plus5V Digital_value': 0.0,
 'ASAP DFE1 Temperature_unit': '°C',
 'ASAP DFE1 Temperature_value': 0.0,
 'ASAP DFE2 Minus5V Analogue_unit': 'V',
 'ASAP DFE2 Minus5V Analogue_value': 0.0,
 'ASAP DFE2 Plus5V Analogue_unit': 'V',
 'ASAP DFE2 Plus5V Analogue_value': 0.0,
 'ASAP DFE2 Plus5V Digital_unit': 'V',
 'ASAP DFE2 Plus5V Digital_value': 0.0,
 'ASAP DFE2 Temperature_unit': '°C',
 'ASAP DFE2 Temperature_value': 0.0,
 'ASAP DFE3 Minus5V Analogue_unit': 'V',
 'ASAP DFE3 Minus5V Analogue_value': 0.0,
 'ASAP DFE3 Plus5V Analogue_unit': 'V',
 'ASAP DFE3 Plus5V Analogue_value': 0.0,
 'ASAP DFE3 Plus5V Digital_unit': 'V',
 'ASAP DFE3 Plus5V Digital_value': 0.0,
 'ASAP DFE3 Temperature_unit': '°C',
 'ASAP DFE3 Temperature_value': 0.0,
 'ASAP DFE4 Minus5V Analogue_unit': 'V',
 'ASAP D

Load the schema:

In [5]:
mySchema = "SEM_schema.json"

with open(mySchema, 'r') as f:
    jsonSchema = json.load(f)
    
readSchema = MetadataSchemaReader(jsonSchema)

schema = readSchema.searchedSchema

Load the map:

In [6]:
with open('new_mapSEM.json', 'r') as m:
    newmapSEM = json.load(m)

Copy the metadata dictionary containing the key-value pairs to be edited:

In [7]:
metadataDict = copy.deepcopy(resultDictionary_filtered_sorted)

Map the terms using `attributeMapping()`:

In [8]:
Map = AttributeMapping(metadataDict, newmapSEM, 'mappedTerms')
workingDict = Map.__dict__

Define a function to clean up the data in the format adhering to the schema:

In [16]:
def cleanData(mappedDict):
    # Make endTime var as ISO 8601
    endTime = mappedDict['entry.endTime.Date'] + ' ' + mappedDict['entry.endTime.Time']
    mappedDict['entry.endTime'] = parser.parse(endTime).isoformat()
    
    # Remove separate date and time vars
    mappedDict.pop('entry.endTime.Date')
    mappedDict.pop('entry.endTime.Time')
    logging.info('Correctly fixed Date')
    
    # Split pixel sizes into x and y
    imgSize = [float(s.strip()) for s in mappedDict['entry.instrument.imaging.numberOfPixels.xPixels'].split('*')]
    mappedDict['entry.instrument.imaging.numberOfPixels.xPixels'] = imgSize[0]
    mappedDict['entry.instrument.imaging.numberOfPixels.yPixels'] = imgSize[1]
    logging.info('Correctly fixed x and y pixels')
    
    # Format tilt correction
    
    tiltCorr = mappedDict['entry.instrument.imaging.tiltCorrection']
    if tiltCorr: 
        mappedDict['entry.instrument.imaging.tiltCorrection'] = ''
    else:
        mappedDict['entry.instrument.imaging.tiltCorrection'] = 'None'
    logging.info('Correctly fixed tiltCorr')
        
    # Deg symbol to "degree"
      
    return {key: value for key, value in sorted(mappedDict.items())}

Define a function to "walk" the nested dict schema and replace the values with their key-value pairs

In [10]:
def modVal(dic, keys, val):
    for key in keys[:-1]:
        dic = dic.setdefault(key, {})
    dic[keys[-1]] = val
    return None

# if complete path in Nicolas' mapping file, will this func work as well?

Define a function that takes the input file and returns an output file:

In [42]:
def workFlow(sourceImg, schema, mapSEM, resultsPath):
    src = sourceImg
    md = zm.zeiss_meta(src)
    del md[0]
    resultDictionary = {**dict((x1+"_value",x2) for x0, x1, x2, x3 in md) , **dict((x1+"_unit",x3) for x0, x1, x2, x3 in md)}
    resultDictionary_filtered = {k: v for k, v in resultDictionary.items() if v != ''}
    metadataDict = dict(sorted(resultDictionary_filtered.items()))
    
    # Open and load schema
    with open(mySchema, 'r') as f:
        jsonSchema = json.load(f)
    
    readSchema = MetadataSchemaReader(jsonSchema)
    schema = readSchema.searchedSchema
    
    # Open and load map
    with open(mapSEM, 'r') as m:
        newmapSEM = json.load(m)
        
    # Map the metadata dictionary
    Map = AttributeMapping(metadataDict, newmapSEM, 'mappedTerms')
    workingDict = Map.__dict__
    
    # Clean up the metadata
    cleanDict = cleanData(workingDict)
        
    # Modify schema file
    outputFile = dict()
    for i, key in enumerate(cleanDict):
        modVal(outputFile, key.split('.'), cleanDict[key])
        
    # Output file to .json
    outputFilename = os.path.join(resultsPath, os.path.basename(src[:-4] + '.json'))
#     outputFilename = os.path.join(resultsPath, src[:-4] + '.json')
    with open(outputFilename, 'w') as f:
        json.dump(outputFile, f)
        
    return outputFile


In [43]:


# imgDir = '/Users/elias/matwerk/sem-mapping/SEM_TIFF_images/'
# resultsPath = '/Users/elias/matwerk/sem-mapping/results'

# pprint(workFlow('FeMoOx_AntiA_04_1k5x_CN.tif', mySchema, myMap, resultsPath))

In [45]:
mySchema  = "SEM_schema.json"
myMap       = 'new_mapSEM.json'
imgDir      = '/Users/elias/Documents/SEM_Mapping_Tool/sem-mapping/test_images/DifferentDetector'
resultsPath =  '/Users/elias/Documents/SEM_Mapping_Tool/sem-mapping/results'

for file in os.listdir(imgDir):
    if file.endswith(".tif"):
        workFlow(os.path.join(imgDir, file), mySchema, myMap, resultsPath)

